# What is `grad_outputs` in the `torch.autograd.grad` function?

Physics-informed neural networks (PINNs) in PyTorch use the `torch.autograd.grad` function to compute the model derivatives that comprise the loss function. `torch.autograd.grad` takes a keyword argument called `grad_outputs` which is needed for PINNs because it specifies the linear combination of model derivatives returned by `torch.autograd.grad`. However, the `grad_outputs` kwarg is not precisely defined in the [documentation](https://pytorch.org/docs/stable/generated/torch.autograd.grad.html). There are many questions and discussions about `grad_outputs` on various online forums, but none that I've found have a clear mathematical definition of `grad_outputs` nor of precisely how it's used by `torch.autograd.grad`. The purpose of these notes is to provide a clear mathematical explanation of `grad_outputs`.

Consider the following tensors:

1. Tensor $X$ with components $X_{bq}$, where $b\in\{0,...,B-1\}$ and $q\in\{0,...,Q-1\}$. For PINNs, $X$ contains the list of collocation points in one batch, specifically $X_{b:}\equiv(X_{b0},...,X_{b(Q-1)})$ is the $b$th collocation point.
2. Tensor $Y=Y(X)$ with components $Y_{b'n}$, where $b'\in\{0,...,B'-1\}$ and $n\in\{0,...,N-1\}$. This will be the function of $X$ whose derivative(s) we are computing. For PINNS, typically $Y$ is the model and $B=B'$ (hence the same letter $B$), and $B$ corresponds to the batch dimension (i.e. $B$ is the number of collocation points).
3. Tensor $V$ with the same shape as $Y$, which will be the value of `grad_outputs`. Often, `V=torch.ones_like(Y)`.

Let `res` be the tensor defined by the code: `res = torch.autograd.grad(Y,X,grad_outputs=V)[0]`

The components of `res` are given by

$$
\texttt{res}_{bq} = \sum_{b'=0}^{B'-1}\sum_{n=0}^{N-1} V_{b'n} \frac{\partial Y_{b'n}}{\partial X_{bq}}
$$

When dealing with batches of collocation points, $B=B'$ and $Y_{bn}(X) = f_n(X_{b:})$ for a function $f:\mathbb{R}^Q\to \mathbb{R}^N$. For PINNs, $f$ may be the model (which, for the $b$th element in the batch, takes the input $X_{b:}$), and $Y$ is the tensor containing the model's output for every element in the batch. In this case,

$$
\frac{\partial Y_{b'n}}{\partial X_{bq}} = \delta_{b'b}\frac{\partial}{\partial X_{bq}}f_n(X_{b:})
$$

Therefore,

$$
\texttt{res}_{bq} = \sum_{n=0}^{N-1} V_{bn} \frac{\partial}{\partial X_{bq}}f_n(X_{b:})
$$

For a typical PINN, $f$ is a scalar-valued function (i.e. $N=1$) and `V=torch.ones_like(Y)`. Hence, in this case,

$$
\texttt{res}_{bq} = \frac{\partial}{\partial X_{bq}}f(X_{b:})
$$

which is the derivative needed to compute the ODE ($Q=1$) or PDE ($Q>1$) loss.